In [1]:
import xml.etree.ElementTree as ET
import re
from nltk.corpus import stopwords
import pandas as pd
import glob
from tqdm import tqdm
from xmldiff import main
import os

STOPWORDS = stopwords.words('english')

In [2]:
class clean():
    """the class for clean purpose"""
    def __init__(self, infile_brd, infile_table, outfile_brd, outfile_table):
        self.infile_brd = infile_brd
        self.infile_table = infile_table
        self.outfile_brd = outfile_brd
        self.outfile_table = outfile_table
        self.var_phrase_map = dict()
        self.var_name_map = dict()
        self.table_new = None
        self.table_new_index_list = None

    def clean_name(self, s, convert_to_lower=True):
        """create name for variables in .brd"""
        s = re.sub('<[^<]+?>', '', s) # markup
        s = re.sub('[^0-9a-zA-Z_\s]', '', s) # keep alnum
        s = re.sub('\t\n\r', '', s) # remove tab, line break, carriage return
        s = ' '.join(s.split()) # remove redundant whitespace
        return s.lower() if convert_to_lower else s
    
    def clean_phrase(self, s, convert_to_lower=False):
        """remove unnecessary part in value"""
        if(s is None or s == ''):
            return None
        s = re.sub('\t\n\r', '', s) # remove tab, line break, carriage return
        s = ' '.join(s.split()) # remove redundant whitespace
        return s.lower() if convert_to_lower else s

    def find_hash(self, s):
        """find hash-like variable"""
        if(s is None or s == ''):
            return False
        # replace "%(" and "%)" to detect whether the variable is a hash-like
        s = re.sub('%\(', '', s) # "\(" is for re to search "("
        s = re.sub('\)%', '', s)
        return s.lstrip('-').isdigit() # ignore "-" in the variable 
    
    def change_var(self, old_name, signature='_', keep_n_words=4):
        """change hash-like variable's name in df"""
        if old_name in self.var_name_map:
            return self.var_name_map[old_name]
        else:
            phrase = self.table_new.loc[old_name].iloc[0] # find the first pharse in the mass production table
            if (old_name is None or old_name == '' or pd.isnull(phrase)):
                return ''
            the_clean_phrase = self.clean_phrase(phrase) 
            h = signature + '_' + '_'.join([word for word in self.clean_name(the_clean_phrase).split(' ') if word not in STOPWORDS][:keep_n_words])
            v = '%(' + str(h) + ')%'
            self.var_name_map[old_name] = v
            self.table_new.rename(index={old_name:v}, inplace=True) # dict key: variable value, dict value: variable name
            return v
    
    def make_var(self, phrase, signature='_', keep_n_words=4):
        """create variable-value pair"""
        if (phrase is None or phrase == ''):
            return ''
        the_clean_phrase = self.clean_phrase(phrase) # clean the value(phrase)
        # if the variable in self.var_phrase_map
        if the_clean_phrase in self.var_phrase_map: 
            return self.var_phrase_map[the_clean_phrase]
        # else create one
        else:
            h = signature + '_' + '_'.join([word for word in self.clean_name(the_clean_phrase).split(' ') if word not in STOPWORDS][:keep_n_words])
            v = '%(' + str(h) + ')%' # create variable name
            self.var_phrase_map[the_clean_phrase] = v # dict key: value, dict value: variable
            return v
        
    def process_txt(self, txt, element, tag, count):
        """process txt"""
        # if txt is empty
        if self.clean_phrase(txt) is None or self.clean_phrase(txt) == '':
            return
        # elif txt is already in the mass production table and it is not a hash-like
        elif txt in self.table_new_index_list and self.find_hash(txt) is False:
            return
        # elif txt is already in the mass production table and it is a hash-like
        elif txt in self.table_new_index_list and self.find_hash(txt) is True:
            if tag == 'Input':
                element[0].text = self.change_var(txt, signature=tag+'_'+str(count))
            else:
                element.text = self.change_var(txt, signature=tag+'_'+str(count))
            return
        # else create a variable name for the value
        else:
            if tag == 'Input':
                element[0].text = self.make_var(txt, signature=tag+'_'+str(count))
            else:
                element.text = self.make_var(txt, signature=tag+'_'+str(count))
            return 
    
    def iterate_generic(self, tag: str, root):
        """replace pharse with variable,
            txt should be %% type or a pharse"""
        count = 1
        for element in root.iter(tag):
            # print(tag)
            if tag == 'Input' and element[0].tag == 'value': # find input value
                txt = element[0].text
                self.process_txt(txt, element, tag, count)
            else:
                txt = element.text
                self.process_txt(txt, element, tag, count)
            count += 1

    def clean_file(self):
        """read the tags and call all functions above"""
        tree = ET.parse(self.infile_brd)
        print("mass production brd input read")
        print("path: " + self.infile_brd)
        root = tree.getroot()

        self.table_new = pd.read_csv(self.infile_table, sep="\t", index_col=0, keep_default_na=False)
        self.table_new_index_list = self.table_new.index.tolist()
        print("mass production table input read")
        print("path: " + self.infile_table)

        tags = ['hintMessage', 'successMessage', 'buggyMessage', 'label', 'Input']
        for tag in tags:
            self.iterate_generic(tag, root) 

        # create new dataframe and concat it with the latest mass production table
        df_new = pd.DataFrame(self.var_phrase_map.keys(), index = list(self.var_phrase_map.values()))
        df_dup = pd.concat([df_new.T]*len(self.table_new.columns)).T
        df_dup.columns = self.table_new.columns
        df_mix = pd.concat([self.table_new, df_dup])
        df_mix.index.name = self.table_new.index.name

        # export the csv
        df_mix.to_csv(self.outfile_table, encoding="utf-8", sep="\t")
        print("mass production table output finished")
        print("path: " + self.outfile_table)

        # export the brd
        tree.write(self.outfile_brd)
        print("mass production brd output finished")
        print("path: " + self.outfile_table)

        return self.table_new, df_mix

In [3]:
class mass_produce:
    """the class for mass produce purpose"""
    def __init__(self, infile_brd, infile_table, outfile_folder):
        self.infile_brd = infile_brd
        self.infile_table = infile_table
        self.outfile_folder = outfile_folder

    def replace_var(self):
        """replace variable with value in the latest mass production table"""
        table_clean = pd.read_csv(self.infile_table, sep="\t", index_col=0, keep_default_na=False)
        for column in range(table_clean.shape[1]):
            for row in range(table_clean.shape[0]):
                content_new = str(table_clean.iloc[row, column])
                start_pattern = "%\("
                # count the number of the replacement in one variable(content_new)
                count = [match.start() for match in re.finditer(start_pattern, str(content_new))]
                for _ in range(len(count)):
                    # find the variable
                    start_pattern = "%\("
                    start_index = [match.start() for match in re.finditer(start_pattern, str(content_new))]
                    end_pattern = "\)%"
                    end_index = [match.start() for match in re.finditer(end_pattern, str(content_new))]
                    # no need for replacement
                    if start_index == []:
                        continue
                    else:
                        variable = content_new[start_index[0]: end_index[0]+2]
                        # find the corresponding column name, and then find the value
                        column_name = table_clean.columns[column]
                        try:
                            value = table_clean.loc[variable, column_name] 
                            # print(variable, value)
                            content_new = content_new.replace(variable, value)
                            table_clean.iloc[row, column] = content_new
                        except:
                            print(variable + " doesn't exist")
        return table_clean
    
    def mass_produce_file(self):
        """iterate and mass produce all the brds"""
        table_clean = self.replace_var()
        for i in range(len(table_clean.columns)):
            column_name = table_clean.columns[i]
            fout = self.outfile_folder + str(table_clean.columns[i]) + ".brd"
            count_line = 0
            count_text = 0
            with open(self.infile_brd, 'r') as infile, open(fout, 'w+') as outfile:
                for line in infile:
                    line = line.replace('\r', '')
                    line_str = str(line)
                    # replace massproduce
                    start_pattern_problem_name = "<ProblemName>"
                    start_index_problem_name = [match.start() for match in re.finditer(start_pattern_problem_name, line_str)]
                    end_pattern_problem_name = "</ProblemName>"
                    end_index_problem_name = [match.start() for match in re.finditer(end_pattern_problem_name, line_str)]
                    start_index_problem_name, end_index_problem_name
                    try:
                        problem_name_old = line_str[start_index_problem_name[0]+13: end_index_problem_name[0]]
                        line_str = line_str.replace(problem_name_old, column_name)
                    except:
                        pass
                    # replace text in first node
                    if count_text == 0:
                        start_pattern_first_node = '<text>'
                        start_index_first_node = [match.start() for match in re.finditer(start_pattern_first_node, line_str)]
                        end_pattern_first_node = '</text>'
                        end_index_first_node = [match.start() for match in re.finditer(end_pattern_first_node, line_str)]
                        if start_index_first_node != []:
                            count_text += 1
                            text_old = line_str[start_index_first_node[0]+6: end_index_first_node[0]]
                            line_str = line_str.replace(text_old, column_name)
                    # count the number of the replacement in one variable(line_str)
                    start_pattern = "%\("
                    count = [match.start() for match in re.finditer(start_pattern, str(line_str))]
                    if count == []:
                        line_str = line_str
                    else:
                        for _ in range(len(count)):
                            start_pattern = "%\("
                            start_index = [match.start() for match in re.finditer(start_pattern, str(line_str))]
                            end_pattern = "\)%"
                            end_index = [match.start() for match in re.finditer(end_pattern, str(line_str))]
                            if start_index == []:
                                line_str = line_str
                            else:
                                variable = line_str[start_index[0]: end_index[0]+2]
                            try:
                                value = table_clean.loc[variable, column_name] 
                                line_str = line_str.replace(variable, value).replace("<%", "&lt;%").replace("%>", "%&gt;")
                            except:
                                print(variable + " doesn't exist")
                    count_line += 1
                    outfile.write(line_str)
                print(fout.split("/")[-1] + " finished")

In [4]:
class validate():
    """the class for validate purpose"""
    def __init__(self, old_folder, new_folder):
        self.old_folder = old_folder
        self.new_folder = new_folder

    def check(self, old_brd, new_brd):
        """use ET.parse to validate"""
        old = ET.parse(old_brd)
        new = ET.parse(new_brd)
        old_text = old.getroot().itertext()
        new_text = new.getroot().itertext()
        set_old = set(old_text)
        set_new = set(new_text)
        if set_old == set_new:
            res = "True"
        else:
            res = "False"
        return set_old, set_new, res

    def check_xmldiff(self, old_brd, new_brd):
        """use xmldiff to validate"""
        diff = main.diff_files(old_brd, new_brd)
        if len(diff) == 0:
            res = "True"
        else:
            res = "False"
        return diff, res
    
    def validate_file(self):
        fs_brd = glob.glob(self.old_folder + "*")
        for old_brd in tqdm(fs_brd, position=0, leave=True):
            new_brd = self.new_folder + old_brd.split("\\", 1)[-1]
            if os.path.exists(new_brd):
                new_brd = new_brd
            elif os.path.exists(new_brd.replace('Problem', '')):
                new_brd = new_brd.replace('Problem', '')
            elif os.path.exists(new_brd.replace(old_brd.split("\\", 1)[-1], "Problem"+old_brd.split("\\", 1)[-1])):
                new_brd = new_brd.replace(old_brd.split("\\", 1)[-1], "Problem"+old_brd.split("\\", 1)[-1])
            else:
                print(old_brd.split("\\", 1)[-1], " cannot find reference")
                continue
            _, _, res_tree = self.check(old_brd, new_brd)
            _, res_diff = self.check_xmldiff(old_brd, new_brd)
            print(old_brd.split("\\", 1)[-1], res_tree, res_diff)


In [5]:
# clean 6.05
infile_brd_c = "./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new.brd"
infile_table_c = "./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new.txt"
outfile_brd_c = infile_brd_c.replace('/HTML_folder/7.17 HTML/7.17 HTML/MassProduction/', '/Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/').replace('.brd', '_cleaned.brd')
outfile_table_c = infile_table_c.replace('/HTML_folder/7.17 HTML/7.17 HTML/MassProduction/', '/Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/').replace('.txt', '_cleaned.txt')

# mass produce for cleaned 6.05
infile_table_m = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
infile_brd_m = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
outfile_folder_m = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# validate for cleanded 6.05
old_folder = "./HTML_folder/6.05 HTML/6.05 HTML/FinalBRDs/"
new_folder = "./Output_translated_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# run the process function
print("clean task ------")
clean_res = clean(infile_brd_c, infile_table_c, outfile_brd_c, outfile_table_c)
_, _ = clean_res.clean_file()
print("mass produce for clean task ------")
mass_produce_clean_res = mass_produce(infile_brd_m, infile_table_m, outfile_folder_m)
mass_produce_clean_res.mass_produce_file()
print("validate for clean task ------")
validate_clean_res = validate(old_folder, new_folder)
validate_clean_res.validate_file()
print("translate task ------")

mass production brd input read
path: ./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new.brd
mass production table input read
path: ./HTML_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new.txt
mass production table output finished
path: ./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt
mass production brd output finished
path: ./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt
clean task ------
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
%(imgNum)% doesn't exist
1.brd finished
2.brd finished
3.brd finished
4.brd finished
5.brd finished
6.brd finished
7.brd finished
8.brd finished
mass produce for clean task ------


 12%|█▎        | 1/8 [00:00<00:03,  2.12it/s]

1.brd True True


 25%|██▌       | 2/8 [00:00<00:02,  2.12it/s]

2.brd True True


...

6.brd True True


 88%|████████▊ | 7/8 [00:03<00:00,  2.43it/s]

7.brd True True


100%|██████████| 8/8 [00:03<00:00,  2.42it/s]

8.brd True True
validate for clean task ------
